# PANDAS

In [ ]:
import os
import pandas as pd
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/auto-mpg.csv")
#print(df[0:5])
df.dtypes

In [ ]:
#display and not print
display(df[0:5])

## statistics

In [ ]:
# Strip non numerics
df = df.select_dtypes(include=['int64','float'])
headers = list(df.columns.values)
headers
fields = []
for field in headers:
    fields.append({
        'name': field,
        'mean': df[field].mean(),
        'var': df[field].var(),
        'sdev': df[field].std()
    })
    
for field in fields:#list od dictionaries
    print(field)

In [ ]:
df2 = pd.DataFrame(fields) #creating a dataframe,
display(df2)

## missing values

In [ ]:

import os
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])
print(f"horsepower has na? {pd.isnull(df['horsepower']).values.any()}")

print("filling miising values....")
med=df['horsepower'].median()
df['horsepower']=df['horsepower'].fillna(med)
#df=df.dropna() #droping misiing values
print(f"horsepower has na? {pd.isnull(df['horsepower']).values.any()}")


## dealing with outliers very high/small data

In [ ]:
#remove all rows here the specified column is +/- sd standard deviation
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name]-df[name].mean())
                         >=(sd*df[name].std()))]
    df.drop(drop_rows, axis = 0, inplace = True)

In [ ]:
import pandas as pd
import os 
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

print(f'length befor droping {len(df)}')
remove_outliers(df,'mpg',2)
print(f'length after droping {len(df)}')

df

## droping fields

In [ ]:
import pandas as pd
import os 

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

df.drop('name', 1, inplace=True)
df

## linking rows and columns

In [ ]:
import pandas as pd
import os 

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

col_horse = df['horsepower']
col_name = df['name']
result = pd.concat([col_horse, col_name], axis = 1)
result

In [ ]:
import pandas as pd
import os 

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])


result = pd.concat([df[0:2], df[-2:]], axis = 1)
result

## spliting into training and validation data

In [ ]:
import pandas as pd
import os 

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

df = df.reindex(np.random.permutation(df.index))# shufling
mask = np.random.rand(len(df))<0.8
trainDF = pd.DataFrame(df[mask])
validationDF = pd.DataFrame(df[~mask])
print(len(trainDF))
print(len(validationDF))

## converting dataframe into a matrix

In [ ]:
df.values 

In [ ]:
df[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values

## saving dataframe to csv


In [ ]:
import pandas as pd
import os 
import numpy as np

path = "."
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

filename_write = os.path.join(path, "auto-mpg-shuffle.csv")
df = df.reindex(np.random.permutation(df.index))# shufling
df.to_csv(filename_write, index=False)

In [ ]:
# as pickle (tip to not loose index)
import os
import pandas as pd
import numpy as np
import pickle

path = "."

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

filename_write = os.path.join(path, "auto-mpg-shuffle.pkl")
df = df.reindex(np.random.permutation(df.index))

with open(filename_write,"wb") as fp:
    pickle.dump(df, fp)

In [ ]:

import os
import pandas as pd
import numpy as np
import pickle

path = "."

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

filename_read = os.path.join(path, "auto-mpg-shuffle.pkl")

with open(filename_write,"rb") as fp:
    df = pickle.load(fp)

pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 5)
display(df)



The danger is that we are now using the target value for training. This technique will potentially lead to overfitting. The possibility of overfitting is even greater if there are a small number of a particular category. To prevent this from happening, we use a weighting factor. The stronger the weight, the more than categories with a small number of values will tend towards the overall average of y. You can perform this calculation as follows.

In [ ]:
def calc_smooth_mean(df1, df2, cat_name, target, weight):
    # Compute the global mean
    mean = df[target].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(cat_name)[target].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + weight * mean) / (counts + weight)

    # Replace each value by the according smoothed mean
    if df2 is None:
        return df1[cat_name].map(smooth)
    else:
        return df1[cat_name].map(smooth),df2[cat_name].map(smooth.to_dict())

## Encoding Categorical Values as Ordinal

Typically categoricals will be encoded as dummy variables. However, there might be other techniques to convert categoricals to numeric. Any time there is an order to the categoricals, a number should be used. Consider if you had a categorical that described the current education level of an individual.

Kindergarten (0)
First Grade (1)
Second Grade (2)
Third Grade (3)
Fourth Grade (4)
Fifth Grade (5)
Sixth Grade (6)
Seventh Grade (7)
Eighth Grade (8)
High School Freshman (9)
High School Sophomore (10)
High School Junior (11)
High School Senior (12)
College Freshman (13)
College Sophomore (14)
College Junior (15)
College Senior (16)
Graduate Student (17)
PhD Candidate (18)
Doctorate (19)
Post Doctorate (20)
The above list has 21 levels. This would take 21 dummy variables. However, simply encoding this to dummies would lose the order information. Perhaps the easiest approach would be to assign simply number them and assign the category a single number that is equal to the value in parenthesis above. However, we might be able to do even better. Graduate student is likely more than a year, so you might increase more than just one value.
